In [1]:
!pip install ohmeow-blurr -q
!pip install bert-score -q

In [2]:
import pandas as pd
from fastai.text.all import *
from transformers import *
from blurr.data.all import *
from blurr.modeling.all import *

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Google colabs/lemonde_csv_formation.csv', error_bad_lines=False, sep=';')
df = df.dropna().reset_index()

In [4]:
#Clean text
df['content'] = df['content'].apply(lambda x: x.replace('/',''))
df['content'] = df['content'].apply(lambda x: x.replace('\xa0',''))
df.head()

,index,category,content,date,title
0,0,sport,"L’international français Jérémy Ménez va rejoindre le club de Bordeaux en Ligue 1, en provenance du Milan AC, sous réserve de la traditionnelle visite médicale, a annoncé le club aquitain dimanche. Ménez est la troisième recrue des Girondins après le milieu de Monaco, Jérémy Toulalan, et l’attaquant guinéen de Bastia, François Kamano. Bordeaux sort d’une pâle saison et repart avec des ambitions nouvelles et l’entraîneur Jocelyn Gourvennec, qui jouit d’une grosse cote grâce à ses six saisons convaincantes à Guingamp. Âgé de 29 ans, Ménez, qui compte 24 sélections (2 buts) chez les Bleu...",2016-07-31,Ligue 1 : Jérémy Ménez signe aux Girondins de Bordeaux
1,1,societe,"Le cousin d’un des assassins du Père Jacques Hamel à Saint-Etienne-du-Rouvray, identifié comme étant Farid K., 30 ans, a été mis en examen, dimanche 31 juillet, pour « association de malfaiteurs terroriste en vue de la préparation de crimes d’atteinte aux personnes » et placé en détention provisoire, selon le parquet. Cet individu«avait parfaitement connaissance, si ce n’est du lieu et du jour précis, de l’imminence d’un projet d’action violente de son cousin», avait dit le parquet avant sa mise en examen. Originaire de Nancy (Meurthe-et-Moselle), il était le dernier des membres de l’...",2016-07-31,Saint-Etienne-du-Rouvray : le cousin d’un des tueurs mis en examen
2,2,societe,"Si le premier ministre Manuel Valls constate que «l’islam a trouvé sa place dans la République», «face à la montée du djihadisme», il y a, selon lui, «urgence» à «bâtir un véritable pacte» avec la deuxième religion de France, à travers une «Fondation pour l’islam de France», explique-t-il dans une tribune publiée le 31juillet dans Le Journal du dimanche. «L’islam a trouvé sa place dans la République (…) contrairement aux attaques répétées des populismes, à droite et à l’extrême droite», qui ont mis cette religion au «cœur de leur rhétorique du bouc émissaire», déclare M. Valls. Et d’a...",2016-07-31,Valls détaille le « pacte » qu’il veut construire avec l’islam de France
3,3,economie,"Les épargnants français sont choyés. Lundi 1eraoût, le taux de rémunération du Livret A aurait théoriquement dû baisser de 0,75% à 0,50%, selon sa formule de calcul fondée sur l’inflation et les taux des marchés monétaires. Mais il en a été décidé autrement. Le 19juillet, François Villeroy de Galhau, le gouverneur de la Banque de France, a proposé de ne pas appliquer la formule et de maintenir inchangé le taux du Livret A non seulement le 1eraoût, mais aussi le 1erfévrier2017, lors de sa prochaine révision. Le ministère des finances a suivi son conseil, «afin de maintenir un rendement...",2016-07-31,"Livret A : les épargnants sourient, le logement social grimace"
4,4,societe,"Un cordon de CRS a bloqué le cortège qui venait juste de se mettre en marche devant la gare du Nord à Paris. Près de 600 personnes étaient présentes, samedi 30juillet, pour réclamer la «vérité» sur les circonstances de la mort d’Adama Traoré, 24 ans, mort onze jours plus tôt, après son interpellation par la gendarmerie à Beaumont-sur-Oise. L’avocat de la famille, Yassine Bouzrou, a révélé vendredi que, dans les deux rapports d’autopsie qu’il a pu consulter, «l’asphyxie» était présentée comme la cause de la mort. Un élément qui n’avait pas été relevé par le procureur de la République d...",2016-07-31,A la marche pour Adama Traoré : « On veut comprendre ce qu’il s’est réellement passé »


In [5]:
#Select only part of it (makes testing faster)
articles = df.head(50)

In [6]:
#Truncate text to make it fit into the model
#articles['content'] = articles['content'].apply(lambda x: ' '.join(x.split()[:700]))

## Import model and set up data

In [7]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

In [8]:
#text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='summarization');

In [9]:
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, task='summarization',
text_gen_kwargs={'max_length': 248,
 'min_length': 56,
 'do_sample': False,
 'early_stopping': True,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False})

blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader('content'), get_y=ColReader('title'), splitter=RandomSplitter())

In [10]:
dls = dblock.dataloaders(articles, bs=2)

### Training


In [11]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]
        }
    }

In [12]:
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

In [13]:
learn.fit_one_cycle(3, lr_max=3e-5, cbs=fit_cbs)

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time


RuntimeError: ignored

### Generate predictions

In [ ]:
df['content'][0]

In [ ]:
outputs = learn.blurr_generate(df['content'][0], early_stopping=False, num_return_sequences=1)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')